In [ ]:
from setfit import SetFitModel, Trainer, TrainingArguments
import pandas as pd 
from datasets import Dataset
from sentence_transformers.losses import CosineSimilarityLoss
from sklearn.linear_model import LinearRegression
from io import StringIO
import csv

In [ ]:
df_train = pd.read_csv("/kaggle/input/data-nostop-lowercase/out_train_no_stop.csv")
df_test = pd.read_csv("/kaggle/input/data-nostop-lowercase/out_test_no_stop.csv")


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sentence_transformers import SentenceTransformer
from setfit import SetFitModel
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
model_body = SentenceTransformer("kinit/slovakbert-sts-stsb")
model_head = SVC()
model = SetFitModel(model_body, model_head)

In [ ]:
output_dict = {}

In [ ]:
for n_samples in [8,16,32,64]:
    print(n_samples)
    cat_splits = []
    for i in range(6):
        cat_splits.append(df_train[df_train["rating_int"]==i].sample(n_samples,random_state=10))
#         cat_splits.append(df_train[df_train["rating_int"]==i].iloc[:n_samples,:])
    new_df_train = pd.concat(cat_splits)
    train_dataset = Dataset.from_pandas(new_df_train)
    model.config = None
    args = TrainingArguments(
        batch_size=16,
        num_epochs=1,

    )
    trainer = Trainer(
        model = model,
        args=args,
        train_dataset=train_dataset,
        column_mapping={"comment" : "text","rating_int":"label"}
    )
    trainer.train()
    x = trainer.evaluate(test_dataset)
    output_dict[n_samples] = x["accuracy"]
    print(output_dict)